## MisClassified Images for German on Winning Model
### Environment Intialization

In [2]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [3]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split
from Data_Preparation.Data_Preparation_German import LoadTestDataSet

importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Data_Preparation\Data_Preparation_German.ipynb
importing Jupyter notebook from C:\Users\Owner\Desktop\Cairo University\Masters\Semester 1\Deep Learning\Project\traffic_sign_env\Traffic_Sign_Recognition_Detection\Source_Code\Pre_Processing\Pre_Processing.ipynb


## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [4]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
/device:GPU:0
Num GPUs Available:  1


In [5]:
def ReadTestDataSet():
    processedTestDataSet = '../DataSet/Processed_DataSet/GermanTestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/GermanTestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/TestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/GermanTestDataSet.pkl")
    return test_image_array, test_image_labels

In [6]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/GermanTestDataSet.pkl
Done Loading Test DataSet.


In [7]:
print(test_image_array.shape)

(12630, 60, 60, 1)


In [8]:
def load_model():
    Model = tf.keras.models.load_model('../Model/Trained_Models/German_Winning_99_69.h5')
    return Model

In [9]:
model= load_model()

In [10]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=2,  verbose=1, min_delta=1e-4, min_lr=1e-20)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

### Retrieve Misclassified Images in German DataSet

In [11]:
predictions= model.predict(test_image_array)

#### CreateMisClassifiedBatchFileCmd
Function creates .cmd bash file to copy all misclassified images in winning 
german model to MisClassified Images Folder

Resultant Image is Image_predictionofmodel_correctprediction.ppm

In [15]:
def CreateMisClassifiedBatchFileCmd():
    i=0
    testing_dataset_dir="..\\DataSet\\Testing_DataSet\\Final_Test\\Images\\"
    f= open("misclassified_german.cmd","w+")
    for prediction, test_image_label in zip(predictions, test_image_labels):
        if np.argmax(prediction) != test_image_label:
            image=str(i)
            if(len(image)==3):
                image="00"+image
            if(len(image)==4):
                image="0"+image
            image+=".ppm"
            image_final=image+"_"+str(np.argmax(prediction))+"_"+str(test_image_label)+".ppm"
            f.write("copy "+testing_dataset_dir+image+ " MisClassification_German_Images\\"+image_final
                  +"\n")
        i=i+1 
        
    f.close()

In [16]:
CreateMisClassifiedBatchFileCmd()